In [1]:
import json
import folium
from folium.plugins import MarkerCluster
import requests
from time import sleep, time, gmtime
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
import operator

host = "https://957cfe07-d816-4770-9cc0-d7c1b4767886.deepnoteproject.com"


In [2]:
coordinates = {
    "PortaNuova":([45.062644, 7.679183], "Centro"),
    "PortaSusa":([45.070097, 7.663892], "CitTurin"),
    "Nizza": ([45.053922, 7.675599], "SanSalvario"),
    "ValentinoNord":([45.059489, 7.687933],"SanSalvario"),
    "Politecnico": ([45.062241, 7.662453], "Crocetta"),
    "Mediterraneo":([45.061467, 7.657903], "Crocetta"),
    "Ferraris":([45.058324, 7.665919], "Crocetta"),
    "Francia": ([45.076091, 7.656601], "CitTurin"),
    "Inghilterra":([45.076343, 7.668333], "CitTurin")
}

In [64]:
#Gets real time the location of an ambulance noise in the last 2 seconds
end = int(time()*1000)
start = end-2000
response = requests.get(host + '/sensor' + f"?from={start}&to={end}")
print(json.loads(response.content.decode()))

if response.ok:
    responseDict = json.loads(response.content.decode())
    mappa=folium.Map(location=[45.062644, 7.679183], zoom_start=13)
    for sensor in responseDict.keys():
        if len(responseDict[sensor]["timestamps"]) > 0:
            folium.Marker(coordinates[sensor][0], popup=f"<i>{sensor}</i>").add_to(mappa)

else:
    print(response.text)

mappa

{'PortaNuova': {'timestamps': [], 'emergencySirens': []}, 'PortaSusa': {'timestamps': [1681898130771], 'emergencySirens': [1.0]}}


In [41]:
#Gets statistics in terms of number of ambulances
end = int(time()*1000)
start = 0
response = requests.get(host + '/sensor' + f"?from={start}&to={end}")

if response.ok:
    responseDict = json.loads(response.content.decode())
    is_ambulance = pd.DataFrame()

    for sensor in responseDict.keys():
        if len(responseDict[sensor]["timestamps"]) > 0:
            for ts, es in zip(responseDict[sensor]["timestamps"],responseDict[sensor]["emergencySirens"]):
                is_ambulance = is_ambulance.append({
                "sensor": sensor,
                "timestamps": ts,
                "emergencySirens": es,
            }, ignore_index = True)
        #Manipulation to add data and set representation granularity in time
        is_ambulance['quartiere'] = is_ambulance["sensor"].map(lambda s: coordinates[s][1])
        is_ambulance['year'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_year)
        is_ambulance['month'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_mon)
        is_ambulance['day'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_mday)
        is_ambulance['year'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_year)
        is_ambulance['weekday'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_wday)
        is_ambulance['hour'] = is_ambulance["timestamps"].map(lambda t: gmtime(t/1000).tm_hour)


else:
    print(response.text)

In [21]:
DeepnoteChart(is_ambulance, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"axis":{"title":null},"sort":null,"type":"nominal","field":"quartiere","scale":{"type":"linear"}},"y":{"axis":{"title":"Emergency Sirens"},"sort":null,"type":"quantitative","field":"emergencySirens","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"sensor","scale":{"scheme":"tableau10"}},"xOffset":{"sort":null,"type":"nominal","field":"sensor"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [22]:
#Delete and check sensors content
#requests.delete(host+"/delete")
dictionary = requests.get(host + "/list")
print(json.loads(dictionary.content.decode()))

{'sensor': ['PortaNuova', 'PortaSusa']}


In [23]:
def zone_min_max_ambulances(start, end, min_max):
    '''gets a timerange and displays the location with the lowest or highest (min_max) number of emergencySirens'''
    response = requests.get(host + '/sensor' + f"?from={start}&to={end}")
    if response.ok:
        responseDict = json.loads(response.content.decode())
        sensor_ambulances={}
        for sensor in responseDict.keys():
            num_ambulances = sum(responseDict[sensor]['emergencySirens'])
            sensor_ambulances[sensor]=num_ambulances
        
        if(min_max=='max'):
            x = max(sensor_ambulances.items(), key=operator.itemgetter(1))
            mappa=folium.Map(location=coordinates[x[0]][0], zoom_start=15)
            folium.Marker(coordinates[x[0]][0], popup=f"<i>{x[1]}</i>",icon=folium.Icon(color="red")).add_to(mappa)
        else:
            x= min(sensor_ambulances.items(), key=operator.itemgetter(1))
            mappa=folium.Map(location=coordinates[x[0]][0], zoom_start=15)   
            folium.Marker(coordinates[x[0]][0], popup=f"<i>{x[1]}</i>",icon=folium.Icon(color="green")).add_to(mappa)      
    else:
        print(response.text)
    return mappa


In [25]:
zone_min_max_ambulances(0, int(time()*1000),'max')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=957cfe07-d816-4770-9cc0-d7c1b4767886' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>